### 1) Divide and Conqure problem:
#### Find the closest pair of points in a plane

In [11]:
def dist(a, b):
    return (((a[0]-b[0])**2) +((a[1]-b[1])**2))**0.5

In [12]:
def closest_brute_force(points):
    min_dist = float("inf")
    p1 = None
    p2 = None
    for i in range(len(points)):
        for j in range(i+1, len(points)):
            d = dist(points[i], points[j])
            if d < min_dist:
                min_dist = d
                p1 = points[i]
                p2 = points[j]
    return p1, p2, min_dist

In [24]:
def closest(points):
    x_sorted = sorted(points, key=lambda point: point[0])
    y_sorted = sorted(points, key=lambda point: point[1])
    
    def _recursive_find(x, y):
        n = len(x)
        if n<=3:
            return closest_brute_force(x)
        else:
            midpoint = x[n//2]
            x_left = x[:n//2]
            x_right = x[n//2:]
            y_left = []
            y_right = []
            for point in y:
                y_left.append(point) if (point[0]<=midpoint[0]) else y_left.append(point)
                
            (p1_left, p2_left, d_l) = _recursive_find(x_left, y_left)
            (p1_right, p2_right, d_r) = _recursive_find(x_right, y_right)
            
            (p1, p2, d) = (p1_left, p2_left, d_l) if (d_l<d_r) else (p1_right, p2_right, d_r)
            
            reduced_in_x = [point for point in y if midpoint[0] - d < point[0] < midpoint[0]+d]
            for i in range(len(reduced_in_x)):
                for j in range(i+1, min(i+7, len(reduced_in_x))):
                    d_new = dist(reduced_in_x[i], reduced_in_x[j])
                    if d_new < d:
                        (p1, p2, d) = (reduced_in_x[i], reduced_in_x[j], d_new)
        return p1, p2, d
    
    return _recursive_find(x_sorted, y_sorted)

In [25]:
test_points = [(2,3), (12, 30), (40, 50), (5,1), (12,10), (3,4)]

In [26]:
print(closest(test_points))

((2, 3), (3, 4), 1.4142135623730951)
